In [1]:
from typing import Callable, Set, Generator, Iterable, Dict, List
import datetime
import collections

import pymysql
import functools
from lxml import etree as et
import httplib2
from tqdm import tqdm

import helpers
import boe

In [2]:
db_host = 'localhost'
db_database = 'boe'
db_user = 'root'
db_password = 'pass'

use_db_settings = lambda func, *args: func(db_host, db_user, db_password, db_database, *args)

# BOE entry mentions extractions

## Helper functions

In [3]:
def retrieve_entries(db_host: str, 
                     db_user: str, 
                     db_password: str, 
                     db_database: str, 
                     day: datetime.datetime) -> Iterable[Dict]:
    '''Retrieve entries for a specific day.'''
    
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password,
                                 db=db_database)

    with connection:
        cursor = connection.cursor(pymysql.cursors.DictCursor)
        cursor.execute(f"SELECT * FROM boe_diary_entry WHERE date = '{day.isoformat()}';")
        entries = cursor.fetchall()
    
    return entries

def get_labels_from_tree(tree):
    tree_search = helpers.use_tree_for_search(tree)
    topics = tree_search(boe.EntryXpath.topics)
    alerts = tree_search(boe.EntryXpath.alerts)
    
    labels = helpers.pipe(topics + alerts,
        functools.partial(map, lambda x: x.xpath('text()')),
        functools.partial(map, ''.join),
        set)
    return labels

def get_tree(item):
    response = helpers.fetch_page('https://www.boe.es' + items[0].get('xml_url'))
    tree = helpers.tree_from_response(response)
    
    return tree

def labels_insert_queries(entry_id: str, labels: Iterable[str]) -> List[str]:
    def single_insert_query(entry_id: str, label: str) -> str:
        return f'''INSERT INTO boe_diary_entry_label (entry, name)
            VALUES ('{entry_id}', '{label}')
        '''
    
    queries = map(functools.partial(single_insert_query, entry_id), labels)
    return queries

def insert_labels(use_db_settings: Callable, entry_id: str, labels: Iterable[str]):
    if len(labels) == 0:
        return
    
    connection = use_db_settings(pymysql.connect)

    with connection:
        cursor = connection.cursor()
        
        for query in labels_insert_queries(entry_id, labels):
            cursor.execute(query)

## Execution

In [6]:
DATE = datetime.datetime(2020, 4, 28)

items = use_db_settings(retrieve_entries, DATE)

for item in tqdm(items):
    tree = get_tree(item)
    labels = get_labels_from_tree(tree)
    insert_labels(use_db_settings, item.get('id'), labels)

0it [00:00, ?it/s]


In [5]:
for item in items:
    tree = get_tree(item)
    labels = get_labels_from_tree(tree)
    print(item.get('title'))
    print(labels)

Resolución de 22 de abril de 2020, del Congreso de los Diputados, por la que se ordena la publicación del Acuerdo de autorización de la prórroga del estado de alarma declarado por el Real Decreto 463/2020, de 14 de marzo.
{'Derecho Constitucional', 'Estado de alarma', 'Enfermedades', 'Epidemias', 'Sanidad'}
Orden INT/368/2020, de 24 de abril, por la que se prorrogan los controles en las fronteras interiores terrestres restablecidos con motivo de la situación de crisis sanitaria ocasionada por el COVID-19.
{'Derecho Constitucional', 'Estado de alarma', 'Enfermedades', 'Epidemias', 'Sanidad'}
Orden INT/369/2020, de 24 de abril, por la que se adoptan medidas excepcionales en materia de formación permanente de los vigilantes de seguridad y vigencia de las tarjetas de identidad profesional del personal de seguridad privada.
{'Derecho Constitucional', 'Estado de alarma', 'Enfermedades', 'Epidemias', 'Sanidad'}
Resolución de 23 de abril de 2020, de la Secretaría General de Industria y de la P

Resolución de la Delegación de Economía y Hacienda en Illes Balears por la que se acuerda el aplazamiento de la subasta pública de bienes muebles convocada para el 28 de mayo de 2020.
{'Derecho Constitucional', 'Estado de alarma', 'Enfermedades', 'Epidemias', 'Sanidad'}
Anuncio de formalización de contratos de: Delegación Especial de la Agencia Tributaria en Andalucía, Ceuta y Melilla. Objeto: Servicio de limpieza integral de los edificios y centros dependientes de la Delegación de la AEAT de Melilla. Expediente: 20560000600.
{'Derecho Constitucional', 'Estado de alarma', 'Enfermedades', 'Epidemias', 'Sanidad'}
Anuncio de formalización de contratos de: Dirección del Servicio de Gestión Económica de la Agencia Estatal de la Administración Tributaria. Objeto: Arrendamiento de la plataforma de detección y prevención de intrusiones de la sede electrónica de la Agencia Estatal de Administración Tributaria. Expediente: 20840022500.
{'Derecho Constitucional', 'Estado de alarma', 'Enfermedades